# 04 – SAM3 / SAM2 Exploration (Google Colab GPU Version)

> **OncoFlow Project** – Brain Tumor Segmentation with SAM3/SAM2

**Goal:** Explore Meta SAM3 (and SAM2 fallback) for brain tumor segmentation on P01 BraTS data.

---

## ⚙️ Setup Requirements
**Before running:** Go to `Runtime → Change runtime type → T4 GPU` (or A100 if Pro)

**Modes tested:**
1. **Point-prompted** – click on tumor centroid (from GT centroid)
2. **Box-prompted** – bounding box of GT tumor region
3. **Automatic** – no user prompt, model generates proposals

**Models compared:**
- SAM3 (`facebook/sam3`) — released Nov 2025 *(requires HuggingFace access request)*
- SAM2 (`facebook/sam2-hiera-large`) — fallback if SAM3 not available
- MedSAM2 (`bowang-lab/MedSAM2`) — medical-adapted variant

---

**📂 Data source:** P01 BraTS NIfTI files + tumor segmentation masks

You have **two options** for data:
- **Option A:** Upload from Google Drive (recommended for real data)
- **Option B:** Generate synthetic MRI phantom data (for pipeline testing)

## 📦 Cell 1: Install Dependencies

This installs SAM2 (stable, widely available) and attempts SAM3 (requires HuggingFace access).

**Expected time:** ~3-5 minutes on first run.

In [ ]:
# ── Cell 1: Install all dependencies ────────────────────────────────────────
import subprocess, sys

def run(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f'STDERR: {result.stderr[-500:]}')
    return result.returncode == 0

print('📦 Installing base scientific packages...')
run('pip install -q nibabel pydicom SimpleITK scipy seaborn pandas matplotlib Pillow')

print('📦 Installing SAM2 from GitHub (stable)...')
run('pip install -q git+https://github.com/facebookresearch/sam2.git')

print('📦 Attempting SAM3 install from GitHub...')
sam3_ok = run('pip install -q git+https://github.com/facebookresearch/sam3.git')
if sam3_ok:
    print('✅ SAM3 package installed (still need HuggingFace model access)')
else:
    print('⚠️  SAM3 pip install failed – will fall back to SAM2')

print('📦 Installing HuggingFace hub...')
run('pip install -q huggingface_hub')

print('\n✅ Installation complete!')

## 🔑 Cell 2: HuggingFace Authentication

Required to download SAM3/SAM2 model weights.

**Steps:**
1. Go to [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Create a **Read** token
3. Paste it below
4. For SAM3 specifically, also request access at [huggingface.co/facebook/sam3](https://huggingface.co/facebook/sam3)

In [ ]:
# ── Cell 2: HuggingFace Authentication ──────────────────────────────────────
from huggingface_hub import login
import os

# Option 1: Paste your token directly (replace the string below)
HF_TOKEN = ''  # ← paste your HF token here, e.g. 'hf_xxxxxxxxxxxx'

# Option 2: Use Colab Secrets (more secure - add via the 🔑 icon in the left sidebar)
if not HF_TOKEN:
    try:
        from google.colab import userdata
        HF_TOKEN = userdata.get('HF_TOKEN')
        print('✅ Using token from Colab Secrets')
    except Exception:
        print('⚠️  No token found in Colab Secrets')

if HF_TOKEN:
    login(token=HF_TOKEN, add_to_git_credential=False)
    print('✅ Logged in to HuggingFace')
else:
    print('⚠️  No HF token provided. Model download may fail for gated models.')
    print('   → For SAM2, no token needed.')
    print('   → For SAM3, request access at https://huggingface.co/facebook/sam3')

## 🖥️ Cell 3: Environment & GPU Check

In [ ]:
# ── Cell 3: Environment setup & GPU verification ─────────────────────────────
import sys, os, time
from pathlib import Path
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import pandas as pd
import seaborn as sns
from scipy.ndimage import binary_erosion

import torch

# Check GPU
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

if DEVICE == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
    print(f'CUDA version: {torch.version.cuda}')
else:
    print('⚠️  No CUDA GPU found! Go to Runtime → Change runtime type → GPU')
    print('    SAM3/SAM2 inference will be VERY slow on CPU.')

# Output directories
BASE_DIR = Path('/content')
OUT_DIR  = BASE_DIR / 'outputs' / '04_sam'
DATA_DIR = BASE_DIR / 'data'
OUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f'\nOutput dir: {OUT_DIR}')
print(f'Data dir:   {DATA_DIR}')

## 📁 Cell 4: Data Setup

Choose **Option A** (Google Drive) or **Option B** (synthetic phantom data).

In [ ]:
# ── Cell 4A: Mount Google Drive (if using real P01 data) ─────────────────────
USE_DRIVE = True   # ← set to False to use synthetic phantom data instead

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    # ── UPDATE THESE PATHS to match your Drive layout ──
    BRATS_DIR = Path('/content/drive/MyDrive/OncoFlow/data/P01/BraTS')
    MASK_DIR  = Path('/content/drive/MyDrive/OncoFlow/data/P01/tumor segmentation')
    # ──────────────────────────────────────────────────

    if BRATS_DIR.exists():
        print(f'✅ BraTS dir found: {BRATS_DIR}')
    else:
        print(f'⚠️  BraTS dir NOT found at: {BRATS_DIR}')
        print('   → Update the path above to match your Drive layout.')
        print('   → Or set USE_DRIVE = False to use synthetic data.')
else:
    print('ℹ️  Skipping Drive mount – will generate synthetic phantom data in Cell 4B.')

In [ ]:
# ── Cell 4B: Generate synthetic MRI phantom data (if no Drive) ──────────────
# This creates realistic-looking synthetic brain MRI volumes with tumor masks
# so you can test the full pipeline without real patient data.

def make_synthetic_brain_volume(shape=(128, 128, 64), seed=42):
    """Generate a synthetic brain MRI volume with a fake tumor."""
    rng = np.random.default_rng(seed)
    vol = np.zeros(shape, dtype=np.float32)

    # Brain ellipsoid
    cx, cy, cz = shape[0]//2, shape[1]//2, shape[2]//2
    rx, ry, rz = shape[0]//2 - 10, shape[1]//2 - 10, shape[2]//2 - 8
    for z in range(shape[2]):
        for y in range(shape[1]):
            for x in range(shape[0]):
                if ((x-cx)/rx)**2 + ((y-cy)/ry)**2 + ((z-cz)/rz)**2 <= 1:
                    vol[x, y, z] = 0.6 + 0.4 * rng.random()

    # Tumor region (bright sphere)
    tx, ty, tz = cx + 20, cy + 10, cz + 5
    tr = 12
    mask = np.zeros(shape, dtype=np.float32)
    for z in range(max(0, tz-tr), min(shape[2], tz+tr)):
        for y in range(max(0, ty-tr), min(shape[1], ty+tr)):
            for x in range(max(0, tx-tr), min(shape[0], tx+tr)):
                if (x-tx)**2 + (y-ty)**2 + (z-tz)**2 <= tr**2:
                    vol[x, y, z] = min(1.0, vol[x,y,z] + 0.5 + 0.3*rng.random())
                    mask[x, y, z] = 1.0

    # Add noise
    vol += rng.normal(0, 0.03, shape).astype(np.float32)
    vol = np.clip(vol, 0, 1)
    return vol, mask

if not USE_DRIVE:
    print('🧠 Generating synthetic brain MRI phantom data...')
    TIMEPOINTS = ['baseline', 'fu1']
    brats_paths = {}
    mask_paths = {}
    affine = np.eye(4)  # 1mm isotropic
    affine[0,0] = affine[1,1] = affine[2,2] = 1.0

    for i, tp in enumerate(TIMEPOINTS):
        vol, mask = make_synthetic_brain_volume(shape=(128, 128, 64), seed=i*100)

        # Save as NIfTI
        tp_dir = DATA_DIR / tp
        tp_dir.mkdir(exist_ok=True)
        mask_dir = DATA_DIR / 'masks'
        mask_dir.mkdir(exist_ok=True)

        t1c_path = tp_dir / 't1c.nii.gz'
        mask_path = mask_dir / f'P01_tumor_mask_{tp}.nii.gz'

        nib.save(nib.Nifti1Image(vol, affine), str(t1c_path))
        nib.save(nib.Nifti1Image(mask, affine), str(mask_path))

        brats_paths[tp] = {'t1c': str(t1c_path)}
        mask_paths[tp] = str(mask_path)

        print(f'  ✅ {tp}: vol={vol.shape}, tumor_voxels={(mask>0).sum()}')

    BRATS_DIR = DATA_DIR
    MASK_DIR  = DATA_DIR / 'masks'
    print('\n✅ Synthetic data ready!')
    print('   Note: Dice scores with synthetic data are meaningless for model evaluation.')
    print('   Use this mode only to test that the pipeline runs end-to-end.')

## 🛠️ Cell 5: Inline Utility Functions

These are self-contained versions of the `utils/` helpers from the local repo, so this notebook is fully standalone in Colab.

In [ ]:
# ── Cell 5: Inline utility functions (standalone – no local utils/ needed) ───

# ── NIfTI helpers ────────────────────────────────────────────────────────────
def load_nifti(path):
    """Load a NIfTI file. Returns (data_array, affine, header)."""
    img = nib.load(str(path))
    return img.get_fdata(dtype=np.float32), img.affine, img.header

def save_nifti(arr, affine, out_path):
    """Save numpy array as NIfTI."""
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    nib.save(nib.Nifti1Image(arr, affine), str(out_path))

# ── Path helpers ─────────────────────────────────────────────────────────────
TIMEPOINTS_ORDER = ['baseline', 'fu1', 'fu2', 'fu3', 'fu4']
MODALITIES = ['t1', 't1c', 't2', 'fla']

def get_p01_brats_paths(data_root):
    root = Path(data_root)
    result = {}
    for tp in TIMEPOINTS_ORDER:
        tp_dir = root / tp
        if tp_dir.exists():
            result[tp] = {
                mod: str(tp_dir / f'{mod}.nii.gz')
                for mod in MODALITIES
                if (tp_dir / f'{mod}.nii.gz').exists()
            }
    return result

def get_p01_mask_paths(data_root):
    root = Path(data_root)
    result = {}
    for tp in TIMEPOINTS_ORDER:
        fpath = root / f'P01_tumor_mask_{tp}.nii.gz'
        if fpath.exists():
            result[tp] = str(fpath)
    return result

# ── Metrics ──────────────────────────────────────────────────────────────────
def dice_coefficient(pred, gt, smooth=1e-6):
    pred = (pred > 0.5).astype(bool)
    gt   = (gt   > 0.5).astype(bool)
    intersection = (pred & gt).sum()
    return float(2.0 * intersection + smooth) / float(pred.sum() + gt.sum() + smooth)

def iou_score(pred, gt, smooth=1e-6):
    pred = (pred > 0.5).astype(bool)
    gt   = (gt   > 0.5).astype(bool)
    intersection = (pred & gt).sum()
    union = (pred | gt).sum()
    return float(intersection + smooth) / float(union + smooth)

def compute_volume_cm3(mask, spacing_mm):
    voxel_vol_mm3 = float(np.prod(spacing_mm))
    n_voxels = int((mask > 0.5).sum())
    return (n_voxels * voxel_vol_mm3) / 1000.0

class Timer:
    def __init__(self, label=''):
        self.label = label
        self.elapsed = 0.0
    def __enter__(self):
        self._start = time.perf_counter()
        return self
    def __exit__(self, *args):
        self.elapsed = time.perf_counter() - self._start
        if self.label:
            print(f'[Timer] {self.label}: {self.elapsed:.3f}s')

class BenchmarkTracker:
    def __init__(self):
        self._rows = []
    def add(self, model, timepoint, pred, gt, spacing, inference_s=0.0, extra=None):
        row = {
            'model': model, 'timepoint': timepoint,
            'dice':  round(dice_coefficient(pred, gt), 4),
            'iou':   round(iou_score(pred, gt), 4),
            'volume_pred_cm3': round(compute_volume_cm3(pred, spacing), 4),
            'volume_gt_cm3':   round(compute_volume_cm3(gt,  spacing), 4),
            'inference_s':     round(inference_s, 2),
        }
        if extra:
            row.update(extra)
        self._rows.append(row)
    def to_dataframe(self):
        return pd.DataFrame(self._rows)

# ── Visualisation ─────────────────────────────────────────────────────────────
sns.set_theme(style='darkgrid', palette='muted')
MASK_ALPHA = 0.45

def _best_slice(mask):
    counts = (mask > 0).sum(axis=(0, 1))
    idx = int(counts.argmax())
    return max(5, min(idx, mask.shape[2] - 6))

def plot_model_comparison(mri, predictions, gt=None, slice_idx=None):
    cols = ['MRI']
    if gt is not None:
        cols.append('GT')
    cols += list(predictions.keys())
    n = len(cols)
    if slice_idx is None:
        slice_idx = _best_slice(gt) if gt is not None else mri.shape[2] // 2
    fig, axes = plt.subplots(1, n, figsize=(n * 3.5, 4))
    if n == 1:
        axes = [axes]
    cmap_list = ['Reds', 'Blues', 'Greens', 'Purples', 'Oranges']
    col_idx = 0
    axes[col_idx].imshow(mri[:, :, slice_idx].T, cmap='gray', origin='lower', aspect='auto')
    axes[col_idx].set_title('MRI (T1c)', fontsize=10)
    axes[col_idx].axis('off')
    col_idx += 1
    if gt is not None:
        axes[col_idx].imshow(mri[:, :, slice_idx].T, cmap='gray', origin='lower', aspect='auto')
        gm = gt[:, :, slice_idx].T
        if gm.any():
            axes[col_idx].imshow(np.ma.masked_where(gm == 0, gm), cmap='Greens',
                                  alpha=MASK_ALPHA, origin='lower', aspect='auto')
        axes[col_idx].set_title('Ground Truth', fontsize=10)
        axes[col_idx].axis('off')
        col_idx += 1
    for (model_name, pred), cmap in zip(predictions.items(), cmap_list):
        axes[col_idx].imshow(mri[:, :, slice_idx].T, cmap='gray', origin='lower', aspect='auto')
        pm = pred[:, :, slice_idx].T
        if pm.any():
            axes[col_idx].imshow(np.ma.masked_where(pm == 0, pm), cmap=cmap,
                                  alpha=MASK_ALPHA, origin='lower', aspect='auto')
        dice = dice_coefficient(pred, gt) if gt is not None else 0.0
        axes[col_idx].set_title(f'{model_name}\nDice={dice:.3f}', fontsize=9, fontweight='bold')
        axes[col_idx].axis('off')
        col_idx += 1
    fig.suptitle(f'SAM Prompt Mode Comparison – Axial slice {slice_idx}', fontsize=13, fontweight='bold')
    plt.tight_layout()
    return fig

print('✅ Utility functions defined')

## 🤖 Cell 6: Load SAM3 → SAM2 Fallback Chain

Tries SAM3 first (requires HF access approval), falls back to SAM2 automatically.

In [ ]:
# ── Cell 6: Load SAM3 → SAM2 → MedSAM2 fallback chain ──────────────────────
SAM_MODEL_LABEL = None
sam_predictor   = None
STUB_MODE       = False

if DEVICE == 'cpu':
    print('⚠️  No GPU detected – running in STUB mode')
    print('   Go to Runtime → Change runtime type → T4 GPU')
    STUB_MODE = True
else:
    # ── Try SAM3 ──────────────────────────────────────────────────────────────
    try:
        from sam3 import SAM3Predictor
        print('⏳ Loading SAM3 (facebook/sam3)...')
        # NOTE: Use 'facebook/sam3' – 'sam3-hiera-large' variant may not exist yet
        sam_predictor   = SAM3Predictor.from_pretrained('facebook/sam3')
        sam_predictor.model.to(DEVICE)
        SAM_MODEL_LABEL = 'sam3'
        print('✅ SAM3 loaded successfully!')
    except ImportError as e:
        print(f'SAM3 import failed (not installed): {e}')
    except Exception as e:
        print(f'SAM3 load failed: {e}')
        if 'gated' in str(e).lower() or '401' in str(e) or 'access' in str(e).lower():
            print('   ↳ Model is gated – request access at https://huggingface.co/facebook/sam3')

    # ── Try SAM2 (fallback) ────────────────────────────────────────────────────
    if sam_predictor is None:
        try:
            from sam2.sam2_image_predictor import SAM2ImagePredictor
            print('⏳ Loading SAM2 (facebook/sam2-hiera-large)...')
            sam_predictor   = SAM2ImagePredictor.from_pretrained('facebook/sam2-hiera-large')
            sam_predictor.model.to(DEVICE)
            SAM_MODEL_LABEL = 'sam2'
            print('✅ SAM2 loaded (SAM3 fallback)')
        except Exception as e2:
            print(f'SAM2 also failed: {e2}')

    # ── Try MedSAM2 (second fallback) ─────────────────────────────────────────
    if sam_predictor is None:
        try:
            from sam2.sam2_image_predictor import SAM2ImagePredictor
            print('⏳ Loading MedSAM2 (bowang-lab/MedSAM2)...')
            sam_predictor   = SAM2ImagePredictor.from_pretrained('bowang-lab/MedSAM2')
            sam_predictor.model.to(DEVICE)
            SAM_MODEL_LABEL = 'medsam2'
            print('✅ MedSAM2 loaded (medical-adapted SAM2)')
        except Exception as e3:
            print(f'MedSAM2 also failed: {e3}')
            print('⚠️  All models failed – falling back to STUB mode')
            STUB_MODE = True

print(f'\n🏷️  Model: {SAM_MODEL_LABEL or "STUB"} | Stub mode: {STUB_MODE} | Device: {DEVICE}')

## 📂 Cell 7: Load Data Paths

In [ ]:
# ── Cell 7: Load data paths and initialise tracker ───────────────────────────
if USE_DRIVE:
    brats_paths = get_p01_brats_paths(BRATS_DIR)
    mask_paths  = get_p01_mask_paths(MASK_DIR)
# else: brats_paths and mask_paths were set in Cell 4B

tracker = BenchmarkTracker()

print(f'Found {len(brats_paths)} timepoints: {list(brats_paths.keys())}')
print(f'Found {len(mask_paths)} masks: {list(mask_paths.keys())}')

for tp, mods in brats_paths.items():
    print(f'  {tp}: {list(mods.keys())}', '✅' if tp in mask_paths else '⚠️  (no mask)')

## 🔧 Cell 8: SAM Inference Helpers

In [ ]:
# ── Cell 8: Prompt generation and slice-wise SAM inference ───────────────────
from PIL import Image as PILImage

def get_tumor_centroid(mask, slice_idx):
    """Return (x, y) centroid of tumor in axial slice."""
    s = mask[:, :, slice_idx]
    if not s.any():
        return (mask.shape[0] // 2, mask.shape[1] // 2)
    ys, xs = np.where(s > 0)
    return (int(xs.mean()), int(ys.mean()))

def get_tumor_bbox(mask, slice_idx):
    """Return [x0, y0, x1, y1] bounding box of tumor in axial slice."""
    s = mask[:, :, slice_idx]
    if not s.any():
        H, W = s.shape
        return [W // 4, H // 4, 3*W // 4, 3*H // 4]
    ys, xs = np.where(s > 0)
    # Add small padding around the GT bbox to simulate realistic prompting
    PAD = 3
    return [max(0, int(xs.min())-PAD), max(0, int(ys.min())-PAD),
            int(xs.max())+PAD, int(ys.max())+PAD]

def volume_to_rgb(vol_slice):
    """Normalise a 2D MRI slice to uint8 RGB."""
    sl = vol_slice.astype(np.float32)
    sl_norm = ((sl - sl.min()) / (sl.max() - sl.min() + 1e-8) * 255).astype(np.uint8)
    return np.stack([sl_norm] * 3, axis=-1)   # H x W x 3

def slicewise_sam_inference(
    predictor,
    volume,
    gt_mask,
    mode='box',         # 'point' | 'box' | 'auto'
    n_slices_max=30,
    verbose=True,
):
    """
    Run SAM2/SAM3 slice-wise on a 3D MRI volume.
    Returns a 3D binary mask of the same shape as volume.
    """
    H, W, D = volume.shape
    mask_3d = np.zeros((H, W, D), dtype=np.float32)

    # Only process slices that contain tumor (guided by GT)
    tumor_slices = [s for s in range(D) if (gt_mask[:, :, s] > 0).any()]
    slices_to_run = sorted(tumor_slices[:n_slices_max])

    if verbose:
        print(f'  Running {mode} inference on {len(slices_to_run)} tumor slices...')

    for i, s in enumerate(slices_to_run):
        rgb = volume_to_rgb(volume[:, :, s])

        with torch.inference_mode():
            predictor.set_image(rgb)

            if mode == 'point':
                cx, cy = get_tumor_centroid(gt_mask, s)
                masks, scores, _ = predictor.predict(
                    point_coords=np.array([[cx, cy]]),
                    point_labels=np.array([1]),
                    multimask_output=False,
                )
            elif mode == 'box':
                bbox = get_tumor_bbox(gt_mask, s)
                masks, scores, _ = predictor.predict(
                    box=np.array(bbox)[None, :],
                    multimask_output=False,
                )
            else:  # auto – uses automatic mask generator
                try:
                    from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
                    amg = SAM2AutomaticMaskGenerator(predictor.model,
                                                     points_per_side=16,
                                                     pred_iou_thresh=0.85)
                    auto_masks = amg.generate(rgb)
                    if auto_masks:
                        best = max(auto_masks, key=lambda x: x['area'])
                        masks = best['segmentation'][None, :, :]
                    else:
                        masks = np.zeros((1, H, W), dtype=bool)
                except Exception as e:
                    print(f'    Auto mode error on slice {s}: {e}')
                    masks = np.zeros((1, H, W), dtype=bool)

        mask_3d[:, :, s] = masks[0].astype(np.float32)

        if verbose and (i+1) % 5 == 0:
            print(f'    Processed {i+1}/{len(slices_to_run)} slices...')

    return mask_3d

print('✅ SAM inference helpers defined')

## 🚀 Cell 9: Run Inference (All Modes + All Timepoints)

In [ ]:
# ── Cell 9: Run inference or stub for all modes ───────────────────────────────
test_timepoints = list(brats_paths.keys())[:2]   # baseline + fu1
print(f'Running on timepoints: {test_timepoints}')
print(f'Model: {SAM_MODEL_LABEL or "STUB"} | Stub: {STUB_MODE}\n')

for tp in test_timepoints:
    print(f'─── Timepoint: {tp} ───────────────────────────────────────────')
    t1c_path = brats_paths[tp].get('t1c')
    gt_path  = mask_paths.get(tp)

    if not t1c_path or not gt_path:
        print(f'  ⚠️  Missing t1c or mask for {tp}, skipping.')
        continue

    vol, vol_aff, _  = load_nifti(t1c_path)
    gt_arr, _, _     = load_nifti(gt_path)
    spacing          = tuple(float(s) for s in nib.load(gt_path).header.get_zooms()[:3])
    best_sl          = int(gt_arr.sum(axis=(0, 1)).argmax())

    print(f'  Volume shape: {vol.shape}, spacing: {spacing}')
    print(f'  GT voxels: {(gt_arr > 0).sum()}, best slice: {best_sl}')

    if STUB_MODE or sam_predictor is None:
        # ── STUB: simulate predictions with morphological ops ──────────────
        gt_bin = (gt_arr > 0)
        stub_point = binary_erosion(gt_bin, iterations=1).astype(np.float32)
        stub_box   = gt_bin.astype(np.float32)
        stub_auto  = binary_erosion(gt_bin, iterations=4).astype(np.float32)

        for mode_label, stub_pred in [
            ('point_stub', stub_point),
            ('box_stub',   stub_box),
            ('auto_stub',  stub_auto),
        ]:
            tracker.add(
                model=f'{SAM_MODEL_LABEL or "sam"}__{mode_label}',
                timepoint=tp,
                pred=stub_pred,
                gt=gt_arr,
                spacing=spacing,
                inference_s=0.0,
                extra={'is_stub': True},
            )
            print(f'  [STUB] {mode_label}: dice={dice_coefficient(stub_pred, gt_arr):.3f}')

        save_nifti(stub_box, vol_aff, OUT_DIR / f'sam_stub_{tp}_box_pred.nii.gz')

    else:
        # ── REAL INFERENCE ──────────────────────────────────────────────────
        for mode in ['point', 'box']:
            print(f'\n  ▶ Mode: {mode}')
            with Timer(f'{SAM_MODEL_LABEL} {mode} {tp}') as t:
                pred = slicewise_sam_inference(
                    sam_predictor, vol, gt_arr,
                    mode=mode, n_slices_max=30, verbose=True,
                )
            dice = dice_coefficient(pred, gt_arr)
            iou  = iou_score(pred, gt_arr)
            print(f'  Dice={dice:.4f} | IoU={iou:.4f} | Time={t.elapsed:.1f}s')

            tracker.add(
                model=f'{SAM_MODEL_LABEL}__{mode}',
                timepoint=tp,
                pred=pred,
                gt=gt_arr,
                spacing=spacing,
                inference_s=t.elapsed,
            )
            save_nifti(pred, vol_aff,
                       OUT_DIR / f'{SAM_MODEL_LABEL}_{tp}_{mode}_pred.nii.gz')

print('\n✅ Inference complete!')
df = tracker.to_dataframe()
print(df[['model', 'timepoint', 'dice', 'iou', 'volume_pred_cm3', 'inference_s']].to_string(index=False))

## 📊 Cell 10: Results Visualisation

In [ ]:
# ── Cell 10: Visualise comparison of prompt modes ─────────────────────────────
tp = 'baseline'
t1c_arr, _, _ = load_nifti(brats_paths[tp]['t1c'])
gt_arr,  _, _ = load_nifti(mask_paths[tp])
sam_label = SAM_MODEL_LABEL or 'sam'

# Collect prediction masks
preds = {}

if STUB_MODE or sam_predictor is None:
    # Load stub predictions
    for mode in ['point_stub', 'box_stub', 'auto_stub']:
        p = OUT_DIR / f'sam_stub_{tp}_box_pred.nii.gz'
        if p.exists():
            arr, _, _ = load_nifti(str(p))
            preds[f'stub_box'] = arr
            break
    # Also generate stubs on-the-fly for comparison
    gt_bin = (gt_arr > 0)
    preds['point_stub'] = binary_erosion(gt_bin, iterations=1).astype(np.float32)
    preds['box_stub']   = gt_bin.astype(np.float32)
    preds['auto_stub']  = binary_erosion(gt_bin, iterations=4).astype(np.float32)
else:
    for mode in ['point', 'box']:
        p = OUT_DIR / f'{sam_label}_{tp}_{mode}_pred.nii.gz'
        if p.exists():
            arr, _, _ = load_nifti(str(p))
            preds[f'{sam_label}_{mode}'] = arr

if preds:
    fig = plot_model_comparison(mri=t1c_arr, predictions=preds, gt=gt_arr)
    plt.savefig(OUT_DIR / f'{sam_label}_modes_comparison.png', dpi=120, bbox_inches='tight')
    plt.show()
    print(f'✅ Saved: {OUT_DIR}/{sam_label}_modes_comparison.png')
else:
    print('⚠️  No prediction files found to visualise.')

In [ ]:
# ── Cell 10B: Metrics bar chart ───────────────────────────────────────────────
df = tracker.to_dataframe()

if not df.empty and 'dice' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Dice
    df_sorted = df.sort_values('dice', ascending=False)
    palette = sns.color_palette('viridis', n_colors=len(df_sorted))
    bars = axes[0].bar(df_sorted['model'] + '\n' + df_sorted['timepoint'],
                       df_sorted['dice'], color=palette, edgecolor='white')
    axes[0].bar_label(bars, fmt='%.3f', padding=3, fontsize=9)
    axes[0].set_title('Dice Score by Model/Mode', fontsize=13, fontweight='bold')
    axes[0].set_ylim(0, 1.15)
    axes[0].set_ylabel('Dice')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].spines[['top', 'right']].set_visible(False)

    # IoU
    df_sorted2 = df.sort_values('iou', ascending=False)
    palette2 = sns.color_palette('plasma', n_colors=len(df_sorted2))
    bars2 = axes[1].bar(df_sorted2['model'] + '\n' + df_sorted2['timepoint'],
                        df_sorted2['iou'], color=palette2, edgecolor='white')
    axes[1].bar_label(bars2, fmt='%.3f', padding=3, fontsize=9)
    axes[1].set_title('IoU Score by Model/Mode', fontsize=13, fontweight='bold')
    axes[1].set_ylim(0, 1.15)
    axes[1].set_ylabel('IoU')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].spines[['top', 'right']].set_visible(False)

    plt.suptitle(f'SAM Model Benchmark – {sam_label}', fontsize=15, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(OUT_DIR / f'{sam_label}_metrics_chart.png', dpi=120, bbox_inches='tight')
    plt.show()

## 💾 Cell 11: Save Results & Download

In [ ]:
# ── Cell 11: Save CSV results and download to local machine ───────────────────
import zipfile

df = tracker.to_dataframe()
csv_path = OUT_DIR / f'{sam_label}_metrics.csv'
df.to_csv(csv_path, index=False)
print(f'✅ Saved metrics: {csv_path}')
print(df[['model', 'timepoint', 'dice', 'iou', 'volume_pred_cm3', 'inference_s']].to_string(index=False))

# Zip outputs for download
zip_path = f'/content/{sam_label}_outputs.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for f in OUT_DIR.glob('*'):
        zf.write(f, f.name)
print(f'\n📦 Outputs zipped: {zip_path}')

# Download the zip (triggers browser download in Colab)
try:
    from google.colab import files
    files.download(zip_path)
    print('⬇️  Download triggered!')
except Exception:
    print(f'  (Not in Colab – find zip at: {zip_path})')

## 📋 Cell 12: Notes & Findings

Fill in after running inference. Copy these results back to your local notebook.

| Item | Finding |
|------|----------|
| GPU used | _(e.g. T4 16GB)_ |
| Model loaded | _(SAM3 / SAM2 / MedSAM2)_ |
| Best prompt mode | _(point / box / auto)_ |
| Dice (box prompt, baseline) | _fill in_ |
| Dice (point prompt, baseline) | _fill in_ |
| Inference time per 30-slice volume | _fill in_ |
| GPU VRAM used | _fill in (check nvidia-smi below)_ |
| **Key finding** | |
| **Recommendation** | Box-prompted SAM2/3 for ensemble; interactive mode for clinical UI |

In [ ]:
# ── Cell 12B: GPU memory usage report ─────────────────────────────────────────
if DEVICE == 'cuda':
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved  = torch.cuda.memory_reserved() / 1e9
    total     = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU Memory:')
    print(f'  Allocated : {allocated:.2f} GB')
    print(f'  Reserved  : {reserved:.2f} GB')
    print(f'  Total VRAM: {total:.2f} GB')

    import subprocess
    result = subprocess.run('nvidia-smi', capture_output=True, text=True)
    print('\nnvidia-smi output:')
    print(result.stdout)
else:
    print('CPU mode – no GPU memory info')